## Open AI gym - MountainCar

### Open AI gym installeerimine

Piisab ka `pip3 install gym`, kuid sedasi ei saa kõiki erinevaid keskkondi alla laadida. Kõikide keskkondade installeerimiseks sobib:

```
git clone https://github.com/openai/gym.git
cd gym
pip3 install -e .
pip3 install -e '.[all]'
```

Rohkem infot: http://gym.openai.com/

In [ ]:
#!pip3 install gym tensorflow numpy keyboard matplotlib

In [11]:
'{nimi1}, {nimi2}'.format(nimi2='märten', nimi1='marie')

'marie, märten'

In [12]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [1]:
x = 5

if functsioon():

else:
    print('midagi muud')

midagi muud


In [9]:
x = 

if x:
    print('if')
else:
    print('else')

else


In [ ]:
import gym

env = gym.make('MountainCar-v0')

- Double DQN (DDQN)
- Dueling DQN
- Distributional RL
- Noisy nets (https://github.com/spring01/drlbox.git)
- Prioritized Experience Replay

**to do**
- N-step Q-Learning

In [ ]:
class SumTree(object):
    data_pointer = 0
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
    
    def add(self, priority, data):
        tree_index = self.data_pointer + self.capacity - 1
        self.data[self.data_pointer] = data
        self.update(tree_index, priority)
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:
            self.data_pointer = 0

    def update(self, tree_index, priority):
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        while tree_index != 0:
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    def get_leaf(self, v):
        parent_index = 0
        while True:
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            else:
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1
        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0]
    
class Memory(object):
    PER_e = 0.01 # min probability
    PER_a = 0.6  # 1-a = pr of selecting randomly (ignoring probability)
    PER_b = 0.4  # effecy of importance sampling (IS) weights on training
    PER_b_increment_per_sampling = 0.001
    absolute_error_upper = 1.

    def __init__(self, capacity):
        self.tree = SumTree(capacity)

    def store(self, experience):
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)
        
    def sample(self, n):
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty(n, dtype=np.float32)
        priority_segment = self.tree.total_priority / n
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            index, priority, data = self.tree.get_leaf(value)
            sampling_probabilities = priority / self.tree.total_priority
            
            b_ISWeights[i] = np.power(n * sampling_probabilities, -self.PER_b) / max_weight
            b_idx[i]= index
            experience = data
            memory_b.append(experience)
            
        return (b_idx, memory_b, b_ISWeights)

    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.python.keras.engine.input_spec import InputSpec
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.keras import backend as K
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import init_ops

class NoisyNet(Layer):
    def __init__(self, **kwargs):
        super(NoisyNet, self).__init__(**kwargs)
        
    def build(self, input_shape):
        
        def make_noise(shape):
            kernel_noise_input = make_fg_noise(shape=[shape[0]])
            kernel_noise_output = make_fg_noise(shape=[shape[1]])
            kernel_noise = kernel_noise_input[:, tf.newaxis] * kernel_noise_output
            return kernel_noise
    
        def make_fg_noise(shape):
            noise = tf.random.normal(shape, dtype=self.dtype)
            trans_noise = tf.sign(noise) * tf.sqrt(tf.abs(noise))
            return tf.Variable(trans_noise, trainable=False, dtype=self.dtype)
        
        input_dim = input_shape[-1]
        kernel_shape = (1, input_dim)
        
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        scale_init = init_ops.Constant(value=(0.25 / np.sqrt(input_dim)))
        
        self.noise_scale = self.add_weight(
            name='noise_scale',
            shape=kernel_shape,
            initializer=scale_init,
            dtype=self.dtype,
            trainable=True
        )
        
        self.noise = make_noise(kernel_shape)
                
    def call(self, inputs):
        training = K.learning_phase()
        
        def add_noise():
            return inputs + self.noise_scale * self.noise
        
        output = tf_utils.smart_cond(training, add_noise, add_noise)#lambda: array_ops.identity(inputs))
        return output

In [ ]:
import numpy as np
import random
from collections import deque
tfk = tf.keras
tfk.backend.clear_session()

class AIAgent:
    """
    A simple neural network based AI agent.
    """
    def __init__(self, train, model_weights_dir=None, use_pretrained=False, memory=Memory(500)):
        self.train = train
        self.train_counter = 0
        self.memory_counter = 0
        self.memory = memory
        self.training_losses = deque(maxlen=10000)
        self.previous_state = None
        self.model_weights_dir = model_weights_dir
        self.use_pretrained = use_pretrained
        
        self.gamma = 0.99     
        self.num_actions = 3
        self.num_atoms = 51
        self.v_max = round(0.7**2,1)
        self.v_min = -0.001
        self.delta_z = (self.v_max - self.v_min) / float(self.num_atoms - 1)
        self.z = [self.v_min + i * self.delta_z for i in range(self.num_atoms)]
        
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.num_samples = 1
        
    def act(self, obs):
        """
        Outputs model actions given the state.
        """
        obs = self.reshape_single_obs(obs)
        z = self.model.predict(obs)
        z_concat = np.vstack(z)
        q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1)
        q = q.reshape(self.num_actions, order='F')
        optimal_action_idxs = np.argmax(q)
        
        return optimal_action_idxs.tolist()
    
    def create_model(self):
        """
        Defines the AI model.
        """
        def ddqn_join(input_layers):
            value = input_layers[0]
            advantage = input_layers[1]
            output = value + tf.subtract(advantage, tf.reduce_mean(advantage, axis=-1, keepdims=True))
            return output
        
        state = tfk.layers.Input(shape=(2), name='state')
        x = tfk.layers.Dense(32, activation=None, name='fully_connected')(state)
        x = NoisyNet(name='noise')(x)
        x = tfk.layers.ReLU(name='activation')(x)
        
        value = tfk.layers.Dense(1, activation=None, name='value')(x)
        advantage = tfk.layers.Dense(self.num_actions, activation=None, name='advantage')(x)
        ddqn = tfk.layers.Lambda(ddqn_join, name='ddqn')([value, advantage])
        
        distribution_list = []
        for i in range(self.num_actions): 
            distribution_list.append(tfk.layers.Dense(self.num_atoms, activation='softmax', name='action_'+str(i))(ddqn))
        
        model = tfk.Model(inputs=state, outputs=distribution_list, name='rainbow')
        model.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.Adam(lr=1e-2))
        
        if self.use_pretrained and self.model_weights_dir:
            model.load_weights(self.model_weights_dir)
            
        return model
        
    def save_memory(self, obs, reward, action, done):
        """
        Save episodes and Q values for training the model.
        """
        obs = self.reshape_single_obs(obs)
        
        if self.previous_state:
                        
            z = self.model.predict(obs)
            z_ = self.target_model.predict(obs)
        
            z_concat = np.vstack(z)
            q = np.sum(np.multiply(z_concat, np.array(self.z)), axis=1)
            q = q.reshape((self.num_samples, self.num_actions), order='F')
            optimal_action_idxs = np.argmax(q, axis=1)
            
            m_prob = [np.zeros((self.num_samples, self.num_atoms)) for i in range(self.num_actions)]
            if done:
                Tz = min(self.v_max, max(self.v_min, reward))
                bj = (Tz - self.v_min) / self.delta_z 
                m_l, m_u = np.floor(bj), np.ceil(bj)
                m_prob[action][0][int(m_l)] += (m_u - bj)
                m_prob[action][0][int(m_u)] += (bj - m_l)
            else:
                for j in range(self.num_atoms):
                    Tz = min(self.v_max, max(self.v_min, reward + self.gamma * self.z[j]))
                    bj = (Tz - self.v_min) / self.delta_z 
                    m_l, m_u = np.floor(bj), np.ceil(bj)
                    m_prob[action][0][int(m_l)] += z_[optimal_action_idxs[0]][0][j] * (m_u - bj)
                    m_prob[action][0][int(m_u)] += z_[optimal_action_idxs[0]][0][j] * (bj - m_l)            
            
            self.memory.store((self.previous_state['obs'], m_prob))
            self.memory_counter += 1
            
        # train model on batch if memory is populated
        if self.train and self.memory_counter >= self.memory.tree.capacity:
            self.train_model()
        
        # reset prev state if episode is completed, else update
        if done:
            self.previous_state = None
        else:
            self.previous_state = {
                'obs': obs,
                'reward': reward,
                'action': action
            }
            
    def train_model(self, batch_size=32):
        """
        Trains the model on minibatch.
        """
        # sample random episides from memory
        b_idx, batch, b_ISWeights = self.memory.sample(batch_size)
                
        self.train_counter += 1
        X = []
        y = []
        for row in batch:
            X.append(row[0])
            y.append(row[1])
        X = np.vstack(X)
        y = np.hstack(y)
        y = [y[i] for i in range(y.shape[0])]
        
        sample_weights = {}
        for i in range(self.num_actions):
            sample_weights['action_'+str(i)] = np.where(np.sum(y[i], axis=1) == 0, 0, b_ISWeights)
        
        train_loss = self.model.train_on_batch(X, y, sample_weight=sample_weights)
        
        pred = self.model.predict(X)
        loss = self.loss(y, pred)
        
        self.memory.batch_update(b_idx, loss)
        self.training_losses.append(train_loss)
        
        if self.model_weights_dir:
            self.model.save(self.model_weights_dir)
        
        if self.train_counter % 1000 == 0:
            self.target_model.set_weights(self.model.get_weights())
            
    def reshape_single_obs(self,obs):
        """
        Reshapes a single observation to fit the model.
        """
        return obs.reshape(1,-1)
    
    def loss(self, y_true, y_pred):
        y_true = np.vstack(y_true).reshape((32, self.num_actions, self.num_atoms), order='F')
        y_pred = np.vstack(y_pred).reshape((32, self.num_actions, self.num_atoms), order='F')
        logloss = -np.sum(np.sum(y_true * np.log(y_pred), axis=-1), axis=-1)
        return logloss

In [ ]:
agent = AIAgent(train=True, model_weights_dir='model_advanced.h5', use_pretrained=False)

for _ in range(50):
    observation = env.reset()
    done = False
    while not done:
        env.render()
        action = agent.act(observation)
        print(action, end='\r')
        observation, reward, done, info = env.step(action)
        
        reward = abs(observation[1])**2
        agent.save_memory(observation, reward, action, done)
        
env.close()

![MountainCar](mountain_car.gif)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 
%config InlineBackend.figure_format = 'png'
plt.style.use('ggplot')

plt.figure(figsize=(8,4))
plt.plot(agent.training_losses)
plt.show()